## Copy Scripts to Current Directory

In [ ]:
# !git clone https://github.com/TheSoundOfAIOSR/rg_sound_generation.git
# !mv /content/rg_sound_generation/tcvae/* .
# !pip install -q ddsp

## Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=False)

## Import Scripts and Setup Config

In [ ]:
from tcvae import localconfig, train

In [ ]:
conf = localconfig.LocalConfig()
# conf.dataset_dir = "/content/drive/MyDrive/the_sound_of_ai/new_data"
conf.batch_size = 1
conf.latent_dim = 16
conf.log_steps = True
conf.learning_rate = 0.001
conf.best_loss = 1

## Train

In [ ]:
train.train(conf)